In [1]:
import tweepy

In [6]:
consumer_key = "z6VGZHRGNwybqWyonTN2QDKb7"
consumer_secret = "j3MatsgQyn7G48RXyjxkbOKpd4TE2QxTNRLiZg0AubEuK63ER2"
access_token = '955033387-u84QPMj9J1pACjcoa369HW3EAwO6NAdC2IwKITE0'
access_token_secret = 'EQwqi2xaCZEWJnhB6dChearbmem4HlMfQHtewQoidCzYu'

# make the connection
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

user = api.me()
print (user.name)

Christina Chang


In [ ]:
API.search